In [31]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import dill
from IPython.core import display as ICD  # to print multiple nice pandas tables
import json
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import tqdm
from tqdm import tqdm_notebook, tnrange
sns.set()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import datasets
import classify
from simple_model import SimpleModel
from testing import test_simple_model

In [33]:
from collections import defaultdict
results = defaultdict(dict)
dump = defaultdict(dict)

In [34]:
import spacy


In [35]:
import tqdm
import numpy as np

nlp_lg = spacy.load('en_vectors_web_lg')

class SpacyWordVectorModel(object):

    def __init__(self, cls=None):
        self.cls = cls

    def fit(self, X, Y):
        embedding = list(map(self.vectorize, tqdm.tqdm_notebook(X)))
        self.stacked = np.vstack(embedding)
        self.cls.fit(self.stacked, Y)
    
    def predict(self, X):
        embedding = list(map(self.vectorize, tqdm.tqdm_notebook(X)))
        Yhat = self.cls.predict(np.vstack(embedding))
        return Yhat
    
    def vectorize(self, x):
        doc = nlp_lg(' '.join(x))
        return doc.vector
    
    def score(self, X, Y):
        Yhat = self.predict(X)
        return 1-((Yhat-Y)**2).mean()
    
    def update(self, X, Y):
        pass

In [36]:
results = defaultdict(dict)

In [37]:
for dataset in datasets.ALL_DATASETS:
    print()
    print(dataset.name())
    model = SpacyWordVectorModel(classify.SkClassifier())
    model.fit(dataset.train_samples(), dataset.train_labels())
    train_p = model.score(dataset.train_samples(), dataset.train_labels())
    test_p = model.score(dataset.test_samples(), dataset.test_labels())
    results[dataset.name()][('logistic', 'train')] = train_p
    results[dataset.name()][('logistic', 'test')] = test_p
    print(train_p, test_p)
    


CRDataset


HBox(children=(IntProgress(value=0, max=3020), HTML(value='')))

HBox(children=(IntProgress(value=0, max=3020), HTML(value='')))

HBox(children=(IntProgress(value=0, max=378), HTML(value='')))


0.8251655629139073 0.8201058201058201

MRDataset


...


0.92225 0.9299999999999999

MPQADataset


HBox(children=(IntProgress(value=0, max=8484), HTML(value='')))

HBox(children=(IntProgress(value=0, max=8484), HTML(value='')))

HBox(children=(IntProgress(value=0, max=1061), HTML(value='')))


0.8915605846298915 0.882186616399623


In [38]:
from sklearn.svm import SVC
for dataset in datasets.ALL_DATASETS:
    print()
    print(dataset.name())
    model = SpacyWordVectorModel(SVC())
    model.fit(dataset.train_samples(), dataset.train_labels())
    train_p = model.score(dataset.train_samples(), dataset.train_labels())
    test_p = model.score(dataset.test_samples(), dataset.test_labels())
    results[dataset.name()][('svc', 'train')] = train_p
    results[dataset.name()][('svc', 'test')] = test_p
    print(train_p, test_p)
    


CRDataset


HBox(children=(IntProgress(value=0, max=3020), HTML(value='')))

HBox(children=(IntProgress(value=0, max=3020), HTML(value='')))

HBox(children=(IntProgress(value=0, max=378), HTML(value='')))


0.6380794701986755 0.6375661375661376

MRDataset


...


0.89625 0.919

MPQADataset


HBox(children=(IntProgress(value=0, max=8484), HTML(value='')))

HBox(children=(IntProgress(value=0, max=8484), HTML(value='')))

HBox(children=(IntProgress(value=0, max=1061), HTML(value='')))


0.8672795851013673 0.8699340245051838


In [39]:
pd.DataFrame(results)

CRDataset  MPQADataset  MRDataset  SUBJDataset
logistic test    0.820106     0.882187   0.750703      0.93000
         train   0.825166     0.891561   0.795169      0.92225
svc      test    0.637566     0.869934   0.708529      0.91900
         train   0.638079     0.867280   0.744284      0.89625

In [40]:
to_file = 'dumps/wordvec_results.pickle'
if True:
    pickle.dump(results, open(to_file, 'wb'))

In [42]:
loaded = pickle.load(open(to_file, 'rb'))

In [43]:
pd.DataFrame(loaded)

CRDataset  MPQADataset  MRDataset  SUBJDataset
logistic test    0.820106     0.882187   0.750703      0.93000
         train   0.825166     0.891561   0.795169      0.92225
svc      test    0.637566     0.869934   0.708529      0.91900
         train   0.638079     0.867280   0.744284      0.89625